In [1]:
import torch

from src.dataset.chess_dataset import ChessDataset, Sizes
from logging_config import setup_logging

# Setup logging
logger = setup_logging()

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# I need to make csv batches of data and uploads it to huggingface
dataset = ChessDataset(size=Sizes.extra_smol)
logger.info("Successfully initialized ChessDataset")

/Users/youcefboumar/PycharmProjects/ChessAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/youcefboumar/PycharmProjects/ChessAI/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


12:26:57 - chessAI - INFO Using device: mps
12:26:57 - chessAI - INFO Initializing ChessDataset
12:26:57 - chessAI - INFO Repository Youcef/chessGames does not exist, creating...
12:26:57 - chessAI - INFO Dataset does not exist, creating...
12:26:58 - chessAI - INFO Read 1000 games. Valid: 64, Invalid: 935, Ratio: 0.07
12:27:00 - chessAI - INFO Read 2000 games. Valid: 148, Invalid: 1851, Ratio: 0.08
12:27:01 - chessAI - INFO Read 3000 games. Valid: 216, Invalid: 2783, Ratio: 0.08
12:27:03 - chessAI - INFO Read 4000 games. Valid: 280, Invalid: 3719, Ratio: 0.08
12:27:04 - chessAI - INFO Read 5000 games. Valid: 356, Invalid: 4643, Ratio: 0.08
12:27:06 - chessAI - INFO Read 6000 games. Valid: 429, Invalid: 5570, Ratio: 0.08
12:27:07 - chessAI - INFO Read 7000 games. Valid: 511, Invalid: 6488, Ratio: 0.08
12:27:09 - chessAI - INFO Read 8000 games. Valid: 582, Invalid: 7417, Ratio: 0.08
12:27:10 - chessAI - INFO Read 9000 games. Valid: 664, Invalid: 8335, Ratio: 0.08
12:27:12 - chessAI - IN

In [2]:
dataset.df_train["Moves"][0]

,Player,Time,Board State
0,0,1.000000,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...
1,1,1.000000,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...
2,0,1.000000,rnbqkbnr/pppp1ppp/4p3/8/4P3/2N5/PPPP1PPP/R1BQK...
3,1,1.000000,rnbqkbnr/ppp2ppp/4p3/3p4/4P3/2N5/PPPP1PPP/R1BQ...
4,0,1.000000,rnbqkbnr/ppp2ppp/4p3/3p4/4P3/2N2N2/PPPP1PPP/R1...
5,1,0.983333,rnbqkb1r/ppp2ppp/4pn2/3p4/4P3/2N2N2/PPPP1PPP/R...
6,0,0.983333,rnbqkb1r/ppp2ppp/4pn2/3pP3/8/2N2N2/PPPP1PPP/R1...
7,1,0.966667,rnbqkb1r/pppn1ppp/4p3/3pP3/8/2N2N2/PPPP1PPP/R1...
8,0,0.983333,rnbqkb1r/pppn1ppp/4p3/3pP3/3P4/2N2N2/PPP2PPP/R...
9,1,0.966667,rnbqkb1r/pp1n1ppp/4p3/2ppP3/3P4/2N2N2/PPP2PPP/...
